<a href="https://colab.research.google.com/github/daniloaleixo/ColabTPUPlayground/blob/master/LTSM_Movie_Sentiment_Classification_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM Movie Sentiment Classification

From tutorial: https://towardsdatascience.com/sentiment-analysis-using-lstm-step-by-step-50d074f09948

In [0]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import os

import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras import models
from keras import layers
import keras

## Load data

Get file

In [0]:
try:
  csv = pd.read_csv("imdb-dataset-of-50k-movie-reviews.zip")
except IOError:
  !wget https://www.dropbox.com/s/pq4yo20ym7fuvc0/imdb-dataset-of-50k-movie-reviews.zip?dl=1 \
  && !mv imdb-dataset-of-50k-movie-reviews.zip\?dl\=1 imdb-dataset-of-50k-movie-reviews.zip \
  && !unzip imdb-dataset-of-50k-movie-reviews.zip

In [0]:
reviews = csv["review"]
labels = csv["sentiment"]

Now we have both dataframes with our reviews and the correspondent sentiment

In [63]:
print(reviews[:10])
print()
print(labels[:10])

0    One of the other reviewers has mentioned that ...
1    A wonderful little production. <br /><br />The...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
4    Petter Mattei's "Love in the Time of Money" is...
5    Probably my all-time favorite movie, a story o...
6    I sure would like to see a resurrection of a u...
7    This show was an amazing, fresh & innovative i...
8    Encouraged by the positive comments about this...
9    If you like original gut wrenching laughter yo...
Name: review, dtype: object

0    positive
1    positive
2    positive
3    negative
4    positive
5    positive
6    positive
7    negative
8    negative
9    positive
Name: sentiment, dtype: object


## Data processing

### Convert to lowercase

In [64]:
reviews = reviews.str.lower()
reviews

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. <br /><br />the...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    i am a catholic taught in parochial elementary...
49998    i'm going to have to disagree with the previou...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object

### Remove HTML tags

In [66]:
import re

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

reviews = reviews.apply(remove_tags)
reviews

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. the filming tec...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    i am a catholic taught in parochial elementary...
49998    i'm going to have to disagree with the previou...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object

### Remove punctuation

In [67]:
from string import punctuation
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [68]:
def remove_punctuations(text):
    for punct in punctuation:
        text = text.replace(punct, '')
    return text

reviews = reviews.apply(remove_punctuations)
reviews

0        one of the other reviewers has mentioned that ...
1        a wonderful little production the filming tech...
2        i thought this was a wonderful way to spend ti...
3        basically theres a family where a little boy j...
4        petter matteis love in the time of money is a ...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    i am a catholic taught in parochial elementary...
49998    im going to have to disagree with the previous...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object